### Read the image

In [3]:
import cv2, numpy as np, os
from PIL import Image
base_path= "../data"
img_path = f"{base_path}/19030958/19030958-0001.png"

img_bgr = cv2.imread(img_path, cv2.IMREAD_COLOR)
img_bgr.shape, img_bgr.dtype



((1700, 2200, 3), dtype('uint8'))

In [15]:
import matplotlib.pyplot as plt, numpy as np, cv2, os, pandas as pd
# Convert to HSV for red mask
hsv = cv2.cvtColor(img_bgr, cv2.COLOR_BGR2HSV)
h,s,v = cv2.split(hsv)

# Red mask (two ranges)
lower1 = np.array([0, 50, 50])
upper1 = np.array([10, 255, 255])
lower2 = np.array([170, 50, 50])
upper2 = np.array([180, 255, 255])
mask_red = cv2.inRange(hsv, lower1, upper1) | cv2.inRange(hsv, lower2, upper2)

# Also mask very light background? We'll keep.
mask_red_ratio = mask_red.mean()/255
mask_red_ratio



np.float64(0.1250548128342246)

In [16]:
img_no_red = img_bgr.copy()
img_no_red[mask_red>0] = (255,255,255)

True

In [17]:
# Create image with red removed by setting those pixels to white
img_no_red = img_bgr.copy()
img_no_red[mask_red>0] = (255,255,255)
# Convert to grayscale
gray = cv2.cvtColor(img_no_red, cv2.COLOR_BGR2GRAY)

# Contrast enhancement using CLAHE
clahe = cv2.createCLAHE(clipLimit=2.0, tileGridSize=(8,8))
gray_eq = clahe.apply(gray)

# Sharpen a bit using unsharp mask
blur = cv2.GaussianBlur(gray_eq, (0,0), 1.0)
sharp = cv2.addWeighted(gray_eq, 1.6, blur, -0.6, 0)

# Threshold to get black curve: use adaptive or Otsu on inverted
# Invert so curve is white on black
inv = 255 - sharp
_, bw = cv2.threshold(inv, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)

# Remove small components and residual text by morphological opening
kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
bw_clean = cv2.morphologyEx(bw, cv2.MORPH_OPEN, kernel, iterations=1)

# Connect broken curve segments slightly (closing with small horizontal emphasis)
kernel_h = cv2.getStructuringElement(cv2.MORPH_RECT, (5,1))
bw_conn = cv2.morphologyEx(bw_clean, cv2.MORPH_CLOSE, kernel_h, iterations=1)

# Find contours
contours, hierarchy = cv2.findContours(bw_conn, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_NONE)

len(contours), sum(cv2.contourArea(c) for c in contours[:10])



error: OpenCV(4.12.0) D:\a\opencv-python\opencv-python\opencv\modules\imgcodecs\src\loadsave.cpp:1051: error: (-2:Unspecified error) could not find a writer for the specified extension in function 'cv::imwrite_'
